### Add alarm configurations, per stream, into the `stream_confgs` table

In [ ]:
from decimal import Decimal

STANDARD_CONFIG = {
    "channels": {
        "email": {"recipients": ["gustavofuhr87@gmail.com"]},
    },
    "rules": {
        "object": {
            "targets": {
                "dog": {
                    "min_confidence": Decimal("0.6"),
                },
                "cat": {
                    "min_confidence": Decimal("0.6"),
                },
                "bird": {
                    "min_confidence": Decimal("0.6"),
                }
            }
        },
        "plate": {
            "targets": {
                "162D11338": {
                    "min_plate_confidence": Decimal("0.7"),
                    "min_ocr_confidence": Decimal("0.7")
                },
            }
        }
    }
}

ANY_MOTION_CONFIG = {
    "channels": {
        "whatsapp": {"numbers": ["+5551996039983"]},
        "email": {"recipients": ["gustavofuhr87@gmail.com"]},
    },
    "rules": {
        "any_motion": {}
    }
}

# "whatsapp": {"numbers": ["+5551996039983"]},

STD_CAMERAS = ["B8A44FE6D078__booth_entrance", "B8A44FB3A1F9__front", "B8A44FE6C853__booth_exit", "B8A44FB981BB__panoramic_trains"]
ANY_MOTION_CAMERAS = ["B8A44FA0DC7A__axis-local"]


In [ ]:
import boto3
from datetime import datetime


dynamodb = boto3.resource("dynamodb", region_name="eu-west-1")
TABLE_NAME = "stream_configs"
table = dynamodb.Table(TABLE_NAME)

def put_ai_config(device_id, config_type, config_data):
    item = {
        "device_id": device_id,
        "config_type": config_type,
        "config": config_data,
        "updated_at": datetime.utcnow().isoformat()
    }

    table.put_item(Item=item)
    print(f"✅ Inserted config for device_id='{device_id}', config_type='{config_type}'")

for cam in STD_CAMERAS:
    put_ai_config(cam, "alarm_config", STANDARD_CONFIG)

for cam in ANY_MOTION_CAMERAS:
    put_ai_config(cam, "alarm_config", ANY_MOTION_CONFIG)
